# Projekt MAGN

### Autorzy:

- Szymon Jurecki
- Dominik Breksa

In [1]:
%pip install .
%pip install pandas
%pip install numpy
%pip install kaggle

Processing c:\users\domin\pycharmprojects\ggsn---magn
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for magn: filename=magn-1.0.0.0-py3-none-any.whl size=15181 sha256=c328466046638a674364201122d5f8d9a31222ff3a3774f4d0d2e33cb7856da1
  Stored in directory: c:\users\domin\appdata\local\pip\cache\wheels\02\69\7a\7a321fc99cdf850f17c5dbe96e53ef1e4b07adda9d5bd3d1f7
Successfully built magn
  Attempting uninstall: magn
    Found existing installation: magn 1.0.0.0
    Not uninstalling magn at c:\users\domin\pycharmprojects\ggsn---magn\src, outside environment C:\Users\domin\PycharmProjects\GGSN--

DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Pobranie oraz Przygotowanie Danych

In [2]:
from pathlib import Path
from typing import Final

resources_dir: Final[Path] = Path('resources/data/')

zip_data_file_movie: Final[Path] = resources_dir.joinpath('movie-data-analytics-dataset.zip')
database_path_movie: Final[Path]  = resources_dir.joinpath('movie.sqlite')

zip_data_file_pitchfork: Final[Path] = resources_dir.joinpath('pitchfork-data.zip')
database_path_pitchfork: Final[Path]  = resources_dir.joinpath('database.sqlite')

Pobranie danych z platformy Kaggle

[Link do danych z platformy Kaggle](https://www.kaggle.com/datasets/shahjhanalam/movie-data-analytics-dataset)

[Link do danych z platformy Kaggle](https://www.kaggle.com/datasets/nolanbconaway/pitchfork-data)

In [3]:
from os.path import exists

if not exists(zip_data_file_movie):
    !kaggle datasets download -d shahjhanalam/movie-data-analytics-dataset -p resources/data
    
if not exists(zip_data_file_pitchfork):
    !kaggle datasets download -d nolanbconaway/pitchfork-data -p resources/data

Rozpakowanie danych do odpowiedniego katalogu

In [4]:
from zipfile import ZipFile

if not exists(database_path_movie):
    with ZipFile(zip_data_file_movie, 'r') as zip_ref:
        zip_ref.extractall(resources_dir)

if not exists(database_path_pitchfork):
    with ZipFile(zip_data_file_pitchfork, 'r') as zip_ref:
        zip_ref.extractall(resources_dir)

Dodanie kluczy głównych i obcych do bazy danych, ponieważ nie zostały one dodane podczas tworzenia na platformie Kaggle, a jest to ważne podczas tworzenia MAGN.

In [5]:
from typing import List, Tuple
import sqlite3

def get_table_columns_types(db_cursor: sqlite3.Cursor, table_name: str) -> List[Tuple[str, str]]:
    """Returns a list of tuples containing the column name and data type of given table in a database."""
    
    info = db_cursor.execute(f"""
        PRAGMA
            table_info({table_name});
    """).fetchall()
    
    return list((table_info[1], table_info[2]) for table_info in info)

def get_table_names(db_cursor: sqlite3.Cursor) -> List[str]:
    """"""
    db_cursor.execute("""
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type='table'
    """)
    
    return list(table[0] for table in db_cursor.fetchall())

def delete_duplicates(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Deletes duplicates from a given table column in a database."""
    
    
    for given_table_name in get_table_names(db_cursor):
        for given_column_name, _ in get_table_columns_types(db_cursor, given_table_name):
            db_cursor.execute(f"""
                DELETE FROM
                    {given_table_name}
                WHERE
                    {given_column_name} IN (
                        SELECT
                            {column_name}
                        FROM
                            {table_name}
                        GROUP BY
                            {column_name}
                        HAVING
                            COUNT(*) > 1
                    );
            """)

def create_table_with_primary_key(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str, column_name: str) -> None:
    """Creates a table with a primary key in a database."""
    
    table_columns = get_table_columns_types(db_cursor, table_name)
    
    db_cursor.execute(f"""
        CREATE TABLE {temporary_table_name} (
            {
                ', '.join(
                    f'{column} {data_type}'
                    if column != column_name
                    else f'{column} {data_type} PRIMARY KEY'
                    for column, data_type in table_columns
                )
            }
        );
    """)
    
def move_data_and_switch_tables(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str) -> None:
    """Moves data from a temporary table to the original table and switches the tables in a database."""
    
    db_cursor.executescript(f"""
        -- Move the data
        INSERT INTO
            {temporary_table_name}
        SELECT
            *
        FROM
            {table_name};
        
        -- Drop the old table
        DROP TABLE
            {table_name};
        
        -- Rename the new table
        ALTER TABLE
            {temporary_table_name}
        RENAME TO
            {table_name};
    """)

def add_primary_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Adds a primary key to a given table column in a database."""
    temporary_table_name = f'{table_name}_'
    
    delete_duplicates(db_cursor, table_name, column_name)
    create_table_with_primary_key(db_cursor, table_name, temporary_table_name, column_name)
    move_data_and_switch_tables(db_cursor, table_name, temporary_table_name)

def create_table_with_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str, temporary_foreign_table_name: str) -> None:
    """Creates a table with a foreign key in a database."""

    foreign_table_columns = get_table_columns_types(db_cursor, foreign_table_name)
    
    db_cursor.execute(f"""
        CREATE TABLE {temporary_foreign_table_name} (
            {
            ', '.join(
                f'{column} {data_type}'
                for column, data_type in foreign_table_columns
            )
            },
            CONSTRAINT fk_{foreign_table_name}_{foreign_column_name}
                FOREIGN KEY
                    ({foreign_column_name})
                REFERENCES
                    {table_name} ({column_name})
        );
    """)

def add_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str) -> None:
    """Adds a foreign key to a given table column in a database."""

    temporary_foreign_table_name = f'{foreign_table_name}_'
        
    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = OFF;
    """)
    
    create_table_with_foreign_key(db_cursor, table_name, column_name, foreign_table_name, foreign_column_name, temporary_foreign_table_name)
    move_data_and_switch_tables(db_cursor, foreign_table_name, temporary_foreign_table_name)

    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = ON;
    """)

In [6]:
with sqlite3.connect(database_path_movie) as conn:
    cursor = conn.cursor()

    cursor.execute(f"""BEGIN;""")
    try:
        add_primary_key(cursor, 'IMDB', 'Movie_id')

        add_foreign_key(cursor, 'IMDB', 'Movie_id', 'earning', 'Movie_id')
        add_foreign_key(cursor, 'IMDB', 'Movie_id', 'genre', 'Movie_id')
    except Exception as error:
        conn.rollback()
        raise error
    else:
        conn.commit()

![ERD Diagram for the database schema](docs/images/database_erd_movie.png)

In [7]:
with sqlite3.connect(database_path_pitchfork) as conn:
    cursor = conn.cursor()

    cursor.execute(f"""BEGIN;""")
    try:
        add_primary_key(cursor, 'reviews', 'reviewid')

        add_foreign_key(cursor, 'reviews', 'reviewid', 'artists', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'content', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'genres', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'labels', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'years', 'reviewid')
    except Exception as error:
        conn.rollback()
        raise error
    else:
        conn.commit()

![ERD Diagram for the database schema](docs/images/database_erd_pitchfork.png)

In [8]:
from magn.database.sqlite3 import get_table_names

all_tables_movie = get_table_names(database_path_movie)

all_tables_movie

['IMDB', 'earning', 'genre']

In [9]:
all_tables_pitchfork = get_table_names(database_path_pitchfork)

all_tables_pitchfork

['reviews', 'artists', 'content', 'genres', 'labels', 'years']

In [10]:
from magn.database.sqlite3 import SQLite3KeysReader

keys_reader = SQLite3KeysReader(database_path_movie, all_tables_movie)
keys_movie = keys_reader.read()

keys_movie

{'IMDB': Keys(primary_keys=['Movie_id'], foreign_keys={}),
 'earning': Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id')}),
 'genre': Keys(primary_keys=[], foreign_keys={'IMDB': ('Movie_id', 'Movie_id')})}

In [11]:
keys_reader = SQLite3KeysReader(database_path_pitchfork, all_tables_pitchfork)
keys_pitchfork = keys_reader.read()

keys_pitchfork

{'reviews': Keys(primary_keys=['reviewid'], foreign_keys={}),
 'artists': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'content': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'genres': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'labels': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'years': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')})}

In [12]:
from magn.database.sqlite3 import SQLite3DataReader

data_reader = SQLite3DataReader(database_path_movie, all_tables_movie, keys_movie)
data_reader.read()

{'IMDB':                                       Title  Rating  TotalVotes MetaCritic  \
 Movie_id                                                                     
 36809               12 Years a Slave (2013)     8.1      496092         96   
 30114                      127 Hours (2010)     7.6      297075         82   
 37367                          50/50 (2011)     7.7      283935         72   
 49473                     About Time (2013)     7.8      225412              
 14867                          Amour (2012)     7.9       76121         94   
 ...                                     ...     ...         ...        ...   
 38262                       Whiplash (2014)     8.5      492285         88   
 10023                 Wreck-It Ralph (2012)     7.7      295125         72   
 27783     X-Men: Days of Future Past (2014)     8.0      560736         74   
 42294             X-Men: First Class (2011)     7.8      556713         65   
 32217                       Zootopia (2016)

In [14]:
data_reader = SQLite3DataReader(database_path_pitchfork, all_tables_pitchfork, keys_pitchfork)
data_reader.read()

{'reviews':                                                       title           artist  \
 reviewid                                                                       
 1                                             young forever        aberfeldy   
 6                                      pure tone audiometry         aarktica   
 7         or you could just go through your whole life a...         aarktica   
 8                                            bleeding light         aarktica   
 10                                                     abcs             abcs   
 ...                                                     ...              ...   
 22721                                          prelapsarian         krallice   
 22722         insecure (music from the hbo original series)  various artists   
 22724                          filthy america its beautiful          the lox   
 22725                                             new start             taso   
 22745           

In [15]:
from magn.database.database import Database

database_movie = Database.from_sqlite3(database_path_movie)

database_movie

Database(all_data={'IMDB': Table(data=                                      Title  Rating  TotalVotes MetaCritic  \
Movie_id                                                                     
36809               12 Years a Slave (2013)     8.1      496092         96   
30114                      127 Hours (2010)     7.6      297075         82   
37367                          50/50 (2011)     7.7      283935         72   
49473                     About Time (2013)     7.8      225412              
14867                          Amour (2012)     7.9       76121         94   
...                                     ...     ...         ...        ...   
38262                       Whiplash (2014)     8.5      492285         88   
10023                 Wreck-It Ralph (2012)     7.7      295125         72   
27783     X-Men: Days of Future Past (2014)     8.0      560736         74   
42294             X-Men: First Class (2011)     7.8      556713         65   
32217                     

In [16]:
database_pitchfork = Database.from_sqlite3(database_path_pitchfork, 1000)

database_pitchfork

Database(all_data={'reviews': Table(data=                                                      title          artist  \
reviewid                                                                      
1                                             young forever       aberfeldy   
6                                      pure tone audiometry        aarktica   
7         or you could just go through your whole life a...        aarktica   
8                                            bleeding light        aarktica   
10                                                     abcs            abcs   
...                                                     ...             ...   
1590                                           constantines    constantines   
1591                                   tournament of hearts    constantines   
1594                                               jane doe        converge   
1595                                            you fail me        converge   
1596       

In [17]:
database_pitchfork.create_mock_target("reviews")

,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,target
reviewid,,,,,,,,,,,,,
1,young forever,aberfeldy,http://pitchfork.com/reviews/albums/1-young-fo...,3.4,0,marc hogan,senior staff writer,2004-08-30,0,30,8,2004,author_type
6,pure tone audiometry,aarktica,http://pitchfork.com/reviews/albums/6-pure-ton...,7.4,0,eric carr,contributor,2003-04-08,1,8,4,2003,author_type
7,or you could just go through your whole life a...,aarktica,http://pitchfork.com/reviews/albums/7-or-you-c...,8.1,0,christopher dare,None,2002-05-16,3,16,5,2002,title
8,bleeding light,aarktica,http://pitchfork.com/reviews/albums/8-bleeding...,7.2,0,sam ubl,None,2005-02-06,6,6,2,2005,best_new_music
10,abcs,abcs,http://pitchfork.com/reviews/albums/10-abcs/,8.0,0,michael bernstein,None,2002-05-07,1,7,5,2002,pub_weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,constantines,constantines,http://pitchfork.com/reviews/albums/1590-const...,8.7,0,dan kilian,None,2001-10-08,0,8,10,2001,title
1591,tournament of hearts,constantines,http://pitchfork.com/reviews/albums/1591-tourn...,7.6,0,marc hogan,senior staff writer,2005-10-10,0,10,10,2005,best_new_music
1594,jane doe,converge,http://pitchfork.com/reviews/albums/1594-jane-...,7.7,0,christopher dare,None,2002-07-02,1,2,7,2002,pub_month
